In [28]:
# 导入库
from bokeh.plotting import figure, output_notebook, show
output_notebook()  # 在notebook中显示
plot = figure()
plot.circle([1,2], [3,4])
show(plot)

Loading BokehJS ...

# 输出为HTML文件

In [4]:
from bokeh.plotting import output_file
output_file("my plot.html", title="my plot html example")

# 输出为Html源码

In [29]:
from bokeh.resources import CDN
from bokeh.embed import file_html
html = file_html(plot, CDN, "my plot")

In [30]:
html

'\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  \n  <head>\n    \n      <meta charset="utf-8">\n      <title>my plot</title>\n      \n      \n        \n          \n        <link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.css" type="text/css" />\n        \n        \n          \n        <script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.js"></script>\n        <script type="text/javascript">\n            Bokeh.set_log_level("info");\n        </script>\n        \n      \n      \n    \n  </head>\n  \n  \n  <body>\n    \n      \n        \n          \n          \n            \n              <div class="bk-root" id="df390b79-3dbd-4144-84b9-fd55c91bad6f" data-root-id="2497"></div>\n            \n          \n        \n      \n      \n        <script type="application/json" id="2787">\n          {"dcede056-1a5d-44d0-8d4a-573a3f7997ae":{"roots":{"references":[{"attributes":{"below":[{"id":"2506","type":"LinearAxis"}],"center":[{"

In [31]:
from IPython.display import HTML
HTML(html) # jupyter中嵌入显示

In [32]:
with open("my plot2.html", "w", encoding='utf-8') as f:
    f.write(html)# 保存为html文件

# 输出为HTML组件

In [33]:
from bokeh.embed import components
plot = figure()
plot.circle([1,2], [3,4])
script, div = components(plot)
print(div )


<div class="bk-root" id="3440c0bf-8f21-4474-8265-a8b8f38ac142" data-root-id="2797"></div>


In [34]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import autoload_static
plot = figure()
plot.circle([1,2], [3,4])
js, tag = autoload_static(plot, CDN, "my_plot.js")  #返回js，将其保存，进而在视图模板中通过js文件进行调用

# 通过web模板显示

In [35]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.sampledata.stocks import AAPL
df = pd.DataFrame(AAPL)
df['date'] = pd.to_datetime(df['date'])
import jinja2  # Flask页面参数传递库
from bokeh.embed import components
# 网页静态模板
template = """
<!DOCTYPE html>
<html lang="en-US">
<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.css" type="text/css" />
<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-widgets-1.1.0.min.css" type="text/css" />   
<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.js"></script>
<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-widgets-1.1.0.min.js"></script>
<body>

    <h1>Hello Bokeh!</h1>
    
    <p> Below is a simple plot of stock closing prices </p>
    
    
     {0}
     {1}

</body>

</html>
"""
p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.line(df['date'], df['close'], color='navy', alpha=0.5)
script, div = components(p)  # 输出组件
html = template.format(script,div)
# 传递组件到静态模板显示图像
from IPython.display import HTML
HTML(html)

In [22]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.sampledata.stocks import AAPL
df = pd.DataFrame(AAPL)
df['date'] = pd.to_datetime(df['date'])
import jinja2  # Flask页面参数传递库
from bokeh.embed import components
# jinja2网页静态模板
template = jinja2.Template("""
<!DOCTYPE html>
<html lang="en-US">
<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.css" type="text/css" />
<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-widgets-1.1.0.min.css" type="text/css" />   
<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.js"></script>
<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-widgets-1.1.0.min.js"></script>
<body>

    <h1>Hello Bokeh!</h1>
    
    <p> Below is a simple plot of stock closing prices </p>
    
    {{ script }}
    
    {{ div }}

</body>

</html>
""")
p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.line(df['date'], df['close'], color='navy', alpha=0.5)
script, div = components(p)  # 输出组件
# 传递组件到静态模板显示图像
from IPython.display import HTML
HTML(template.render(script=script, div=div))

In [36]:
# 动态交互
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.embed import components
# 数据
x = [x*0.005 for x in range(0, 200)]
y = x
source = ColumnDataSource(data=dict(x=x, y=y))
# 画布
plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
# 回调函数
callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")
# 滑动条
slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)
# 图层，列表显示
layout = column(slider, plot)
# show(layout)  # 直接用Notebook显示

In [37]:
script, div = components(layout)
template2="""
<!DOCTYPE html>
<html lang="en-US">

        <link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.css" type="text/css" />
        <link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-widgets-1.1.0.min.css" type="text/css" />   
        <script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-1.1.0.min.js"></script>
        <script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-widgets-1.1.0.min.js"></script>

<body>

    <h1>Hello Bokeh!</h1>   
      
    {0}{1}

</body>

</html>
""".format(script,div)
from IPython.display import HTML
HTML(template2) # 用内置HTML函数显示

In [39]:
from bokeh.resources import CDN
from bokeh.embed import file_html
html = file_html(layout, CDN, "my plot")
from IPython.display import HTML
HTML(html) 

# Bokeh Flask

In [40]:
# 详见目录：flask-bokeh-example

# Bokeh sever

In [25]:
# 将该示例代保存为sliders.py
import numpy as np
from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure
# 数据
N = 200
x = np.linspace(0, 4*np.pi, N)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))
# 画布
plot = figure(plot_height=400, plot_width=400, title="my sine wave",
              tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])
# 绘图
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
# 控件
text = TextInput(title="title", value='my sine wave')
offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)
# 回调
def update_title(attrname, old, new):
    plot.title.text = text.value
text.on_change('value', update_title)
def update_data(attrname, old, new):
    # 获取滑块当前数据
    a = amplitude.value
    b = offset.value
    w = phase.value
    k = freq.value
    # 生成新的数据
    x = np.linspace(0, 4*np.pi, N)
    y = a*np.sin(k*x + w) + b
    source.data = dict(x=x, y=y)
for w in [offset, amplitude, phase, freq]:
    w.on_change('value', update_data)
# 设置显示方式
inputs = column(text, offset, amplitude, phase, freq)
# 服务器托管，win平台在CMD文件路径下运行：bokeh serve sliders.py
# curdoc().add_root(row(inputs, plot, width=800))
# curdoc().title = "Sliders"
# 为便于在notebook展示，将Bokeh数据文档采用如下转换
def fc_doc(doc):
    doc.add_root(row(inputs, plot, width=800))
    doc.title = "Sliders"
show(fc_doc) # 万法皆对象